In [ ]:
import pickle
import datetime
import pandas as pd
import altair as alt

import bok.parsers

In [ ]:
# Define a conversion rate based on the current global market exchange rate.
IDR_TO_USD_RATE = 1.0/14150

## Extracting data from the transactions file

In [ ]:
transactions = bok.parsers.parse_transactions_log("data/clean/transactions.log")

## Each user's total amount of data purchased directly.

In [ ]:
purchases = transactions.loc[transactions["kind"] == "purchase"]
aggregated_purchases = purchases.groupby("user")

plots_frame = aggregated_purchases[["amount_idr", "amount_bytes"]].sum()
plots_frame = plots_frame.reset_index()

plots_frame["amount_GiB"] = plots_frame["amount_bytes"] * float(1)/(1024**3)
print(plots_frame)

In [ ]:
alt.Chart(plots_frame).mark_bar().encode(
    x=alt.X('user',
            sort=alt.SortField(field="amount_bytes",
                               order="descending"
                               ),
            ),
    y=alt.Y('amount_GiB',
            scale=alt.Scale(type="log"),
            ),
).display()

## Each user's count of purchase transactions

In [ ]:
purchases = transactions.loc[transactions["kind"] == "purchase"]
purchase_counts = purchases.groupby("user")["timestamp"].count()

plot_frame = purchase_counts.reset_index().rename({"timestamp": "purchase_count"},
                                                  axis="columns")

alt.Chart(plot_frame).mark_bar().encode(
    x=alt.X('user',
            sort=alt.SortField(field="purchase_count",
                               order="descending"
                               ),
            ),
    y=alt.Y('purchase_count',
            scale=alt.Scale(type="log"),
            ),
).display()


## Each user's count of transfer transactions

In [ ]:
user_transfers = transactions.loc[transactions["kind"] == "user_transfer"]
transfer_counts = user_transfers.groupby("user")["timestamp"].count()

plot_frame = transfer_counts.reset_index().rename({"timestamp": "transfer_src_count"},
                                                  axis="columns")

transfer_dst_counts = user_transfers.groupby("dest_user")["timestamp"].count()

transfer_dst_counts = transfer_dst_counts.reset_index().rename(
    {"timestamp": "transfer_dst_count",
     "dest_user": "user"
    },
    axis="columns"
)

plot_frame = plot_frame.merge(transfer_dst_counts, how="outer")

plot_frame = plot_frame.melt(id_vars=["user"],
                             value_vars=["transfer_src_count", "transfer_dst_count"],
                             var_name="direction",
                             value_name="count")

alt.Chart(plot_frame).mark_bar(opacity=0.7).encode(
    x=alt.X('user',
            sort=alt.SortField(field="count",
                               order="descending"
                               ),
            ),
    y=alt.Y('count',
            scale=alt.Scale(type="log"),
            stack=False,
            ),
    color="direction"
).display()


## Each user's amount of currency transferred

In [ ]:
user_transfers = transactions.loc[transactions["kind"] == "user_transfer"]
transfer_counts = user_transfers.groupby("user")["amount_idr"].sum()

plot_frame = transfer_counts.reset_index().rename({"amount_idr": "transfer_src_sum"},
                                                  axis="columns")

transfer_dst_counts = user_transfers.groupby("dest_user")["amount_idr"].sum()

transfer_dst_counts = transfer_dst_counts.reset_index().rename(
    {"amount_idr": "transfer_dst_sum",
     "dest_user": "user"
    },
    axis="columns"
)

plot_frame = plot_frame.merge(transfer_dst_counts, how="outer")
plot_frame = plot_frame.melt(id_vars=["user"],
                             value_vars=["transfer_src_sum", "transfer_dst_sum"],
                             var_name="direction",
                             value_name="idr")

# Convert to USD
plot_frame["usd"] = plot_frame["idr"] * IDR_TO_USD_RATE

alt.Chart(plot_frame).mark_bar(opacity=0.7).encode(
    x=alt.X('user',
            sort=alt.SortField(field="usd",
                               order="descending"
                               ),
            ),
    y=alt.Y('usd',
            scale=alt.Scale(type="log"),
            stack=False,
            ),
    color="direction"
).display()

## Graph network revenue and expenses vs time

In [ ]:
topups = transactions.loc[transactions["kind"] == "admin_topup"]
topups = topups.set_index("timestamp")[["dest_user", "amount_idr"]]
topups["idr_cumulative"] = topups["amount_idr"].cumsum()

# Convert to USD
topups["usd_cumulative"] = topups["idr_cumulative"] * IDR_TO_USD_RATE

topups = topups.reset_index()

alt.Chart(topups).mark_line().encode(
    x=alt.X('timestamp',
            type="temporal"
            ),
    y=alt.Y("usd_cumulative",
            ),
).display()

## From the transactions data, a graph of the number of transactions of different costs.

In [ ]:
purchases = transactions.loc[transactions["kind"] == "purchase"]
purchases = purchases.groupby("amount_bytes")["timestamp"].count()
purchases = purchases.reset_index().rename({"timestamp": "count"}, axis="columns")
purchases["amount_MB"] = purchases["amount_bytes"] * 1.0/1000**2
purchases["total_GB"] = purchases["amount_MB"] * purchases["count"] * 1.0/1000

print(purchases)
bars = alt.Chart(purchases).mark_bar().encode(
    x=alt.X('amount_MB',
            type="ordinal",
            ),
    y='count',
)

text = bars.mark_text(
    align="left",
    baseline="bottom",
).encode(
    text="count:Q"
)

bars = text + bars

bars.display()

alt.Chart(purchases).mark_bar().encode(
    x=alt.X('amount_MB',
            type="ordinal",
            ),
    y='total_GB',
).display()

## Explore which package most users select

In [ ]:
user_packages = transactions.loc[transactions["kind"] == "purchase"]
user_packages = user_packages.groupby(["user", "amount_bytes"])["timestamp"].count()
user_packages = user_packages.reset_index().rename({"timestamp":"count"}, axis="columns")
user_packages["amount_MB"] = user_packages["amount_bytes"] * 1.0/1000**2

alt.Chart(user_packages).mark_bar(opacity=0.7).encode(
    x=alt.X('user',
            sort=alt.SortField(field="count",
                               order="descending"
                               ),
            ),
    y=alt.Y('count',
            scale=alt.Scale(type="linear"),
            stack="normalize",
            ),
    color="amount_MB:N"
).display()

alt.Chart(user_packages).mark_bar(opacity=0.7).encode(
    x=alt.X('user',
            sort=alt.SortField(field="count",
                               order="descending"
                               ),
            ),
    y=alt.Y('count',
            scale=alt.Scale(type="linear"),
            stack="zero",
            ),
    color="amount_MB:N"
).display()

## Explore the amount of bytes sold at each package level

In [ ]:
bytes_per_package = user_packages
bytes_per_package["bytes_per_package"] = bytes_per_package["count"] * bytes_per_package["amount_bytes"]

print(bytes_per_package)

alt.Chart(bytes_per_package).mark_bar(opacity=0.7).encode(
    x=alt.X('user',
            sort=alt.SortField(field="bytes_per_package",
                               order="descending"
                               ),
            ),
    y=alt.Y('bytes_per_package',
            scale=alt.Scale(type="linear"),
            stack="normalize",
            ),
    color="amount_MB:N"
).display()

alt.Chart(bytes_per_package).mark_bar(opacity=0.7).encode(
    x=alt.X('user',
            sort=alt.SortField(field="bytes_per_package",
                               order="descending"
                               ),
            ),
    y=alt.Y('bytes_per_package',
            scale=alt.Scale(type="linear"),
            stack="zero",
            ),
    color="amount_MB:N"
).display()

In [ ]:
## Purchases vs. Transfers vs. Topups

In [ ]:
categorized_transactions_df = pd.DataFrame({'type': list(categorized_transactions.keys()),
                                           'amount': list(categorized_transactions.values())})

alt.Chart(categorized_transactions_df).mark_bar(size=30).encode(
    x='type',
    y='amount',
    tooltip=['type:N', 'bytes:Q']
).properties(width=100)

## Graph of how much data was bought per day, in total, by the members of the Bokondini community.

In [ ]:
bytes = dict()

for entry in purchases:
    if entry[0] not in bytes:
        bytes[entry[0]] = float(entry[4]) / 1000000000
    else:
        bytes[entry[0]] += float(entry[4]) / 1000000000

In [ ]:
dates = list()
for i in range((datetime.date(2019, 5, 17) - datetime.date(2019, 3, 4)).days + 1):
    dates.append(str(datetime.date(2019, 3, 4) + datetime.timedelta(days=i)))
def load_average(week):
    with open("data/weeks/remote_userlogs/remote_userlog-%s.pickle" % week, mode="rb") as h:
        remote_userlog = pickle.load(h)
    times = dict()
    days = list()
    index = 0
    days.append(remote_userlog[index]['start_time'].date())
    for entry in remote_userlog:
        if entry['start_time'].date() != days[index]:
            index += 1
            days.append(entry['start_time'].date())
        for day in days:
            times[str(day)] = 0
    for entry in remote_userlog:
        times[str(entry['start_time'].date())] += entry['bytes_b_to_a']
        times[str(entry['start_time'].date())] += entry['bytes_a_to_b']
    return times

In [ ]:
total = list()
dates = list()
date = datetime.date(2019, 3, 4)
while (date < datetime.date(2019, 5, 17)):
    print(date)
    total.append(load_average(str(date)))
    date += datetime.timedelta(days=7)

In [ ]:
days = dict()

for week in total:
    for day in week:
        if day not in days.keys():
            days[day] = week[day] / 1000000000
        else:
            days[day] += week[day] / 1000000000
            
total = list()
for entry in bytes:
    total.append(['purchase', entry, bytes[entry]])
    
for entry in days:
    total.append(['use', entry, days[entry]])

In [ ]:
day_avg_df = pd.DataFrame(total)

alt.Chart(day_avg_df).mark_bar(opacity=0.7).encode(
    x=alt.X('1:T', title='date'),
    y=alt.Y('2:Q', stack=None, title='traffic in GB'),
    color=alt.Color('0:N', title='type')
)

In [ ]:
bytes = dict()

for entry in purchases:
    if entry[0] not in bytes:
        bytes[entry[0]] = float(entry[4]) / 1000000000
    else:
        bytes[entry[0]] += float(entry[4]) / 1000000000
        
total_data = pd.DataFrame({'day': list(bytes.keys()),
                          'total data purchased, in GB': list(bytes.values())})

alt.Chart(total_data).mark_bar().encode(
    x='day',
    y='total data purchased, in GB'
)

## Graph of how much data was transferred between users per day, in total, by the members of the Bokondini community

In [ ]:
bytes = dict()

for entry in transfers:
    if entry[0] not in bytes:
        bytes[entry[0]] = float(entry[5].strip()) / 1000000
    else:
        bytes[entry[0]] += float(entry[5].strip()) / 1000000
        
total_data = pd.DataFrame({'day': list(bytes.keys()),
                          'total data transferred, in MB': list(bytes.values())})

alt.Chart(total_data).mark_bar().encode(
    x='day',
    y='total data transferred, in MB'
)

In [ ]:
bytes = dict()

for entry in topups:
    if entry[0] not in bytes:
        bytes[entry[0]] = float(entry[4]) / 1000000
    else:
        bytes[entry[0]] += float(entry[4]) / 1000000
        
total_data = pd.DataFrame({'day': list(bytes.keys()),
                          'total data topped up, in MB': list(bytes.values())})

alt.Chart(total_data).mark_bar().encode(
    x='day',
    y='total data topped up, in MB'
)


In [ ]:
# Check date time handling
import altair as alt
import pandas as pd

df = pd.DataFrame({'local': ['2018-01-01T00:00:00'],
                   'utc': ['2018-01-01T00:00:00Z']})

alt.Chart(df).transform_calculate(
    compliant="hours(datum.local) != hours(datum.utc) ? true : false",
).mark_text(size=20, baseline='middle').encode(
    text=alt.condition('datum.compliant', alt.value('OK'), alt.value('not OK')),
    color=alt.condition('datum.compliant', alt.value('green'), alt.value('red'))
).properties(width=80, height=50)